# Decision Trees

Leitura dos datasets tratados

In [1]:
import lightgbm as lgb
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('../datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('../datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e modelação

In [2]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [3]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': list(range(1,20)), 
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.05, 0.1, 0.3]
}

grid = GridSearchCV(lgb.LGBMClassifier(), param_grid, cv=3)
grid.fit(X_treino, y_treino)

grid.best_score_, grid.best_params_

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001847 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5547
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 31
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -0.559616
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000512 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise

(0.8842857142857143,
 {'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100})

In [5]:
predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/lightBGMPrediction.csv')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
